In [33]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
import os
from time import sleep

In [34]:
!which chromedriver

/usr/local/bin/chromedriver


In [35]:
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

In [36]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

url = 'https://mars.nasa.gov/news/'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

# Examine the results, then determine element that contains sought info
#print(soup.prettify())


In [37]:
news_title = soup.find('div', class_='content_title').text
print(f"Most Recent Title: {news_title}")
news_p = soup.find('div', class_='article_teaser_body')
print(f"First Paragraph: {news_p}")

# NOTE: I've tried muptiple ways and keep getting a null object returned for article teaser body     

Most Recent Title: 

NASA's Opportunity Rover Mission on Mars Comes to End


First Paragraph: None


In [38]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called `featured_image_url`.

# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [39]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [40]:
# Navigate to the page with the full, large image
browser.click_link_by_id('full_image')
sleep(5)
#browser.click_link_by_partial_text('more info')

In [41]:
browser.click_link_by_partial_text('more info')

In [42]:
# use bs to image source
html = browser.html
soup = bs(html, 'html.parser')
image_url = soup.find('img').get('src')
base_url = "https://www.jpl.nasa.gov/spaceimages"
featured_img_url = base_url + image_url
print (f"The featured image's URL is: {featured_img_url}")

The featured image's URL is: https://www.jpl.nasa.gov/spaceimages/assets/images/logo_nasa_trio_black@2x.png


In [43]:
### Mars Weather

# Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) 
# Scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called `mars_weather`.

twitter_wx_url = "https://twitter.com/marswxreport?lang=en"
tweet_response = requests.get(twitter_wx_url)

tweet_soup = bs(tweet_response.text, 'html.parser')
mars_weather = tweet_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print("Today's weather on Mars:")
print(mars_weather)

Today's weather on Mars:
InSight sol 97 (2019-03-05) low -95.3ºC (-139.6ºF) high -14.8ºC (5.3ºF)
winds from the SW at 4.9 m/s (10.9 mph) gusting to 12.9 m/s (28.9 mph)
pressure at 7.20 hPapic.twitter.com/Q9QivIo4ls


In [44]:
### Mars Facts

# Visit the Mars Facts webpage [here](http://space-facts.com/mars/) 
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

mars_facts_url = "http://space-facts.com/mars/"
sleep(3)  
mars_table = pd.read_html(mars_facts_url) 
mars_facts_df = mars_table[0]
mars_facts_df

HTTPError: HTTP Error 500: Internal Server Error

In [45]:
# Use Pandas to convert the data to a HTML table string.

html_table = mars_facts_df.to_html()
print(html_table.translate({ord('\n'): None}))


NameError: name 'mars_facts_df' is not defined

In [46]:
### Mars Hemispheres

# Visit the USGS Astrogeology site [here]:
# (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) 
# to obtain high resolution images for each of Mar's hemispheres.
base_url = "https://astrogeology.usgs.gov/"
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
hemi_html = browser.html
hemi_soup = bs(hemi_html, 'html.parser')
hemi_soup

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
<script async="" src="https://ssl.google-analytics.com/ga.js" type="text/javascript"></script><script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js" type="text/javascript"></script>
<title>Astropedia Search Results | USGS Astrogeology Science Center</title>
<meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verifica

In [65]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

# image_urls empty list to hold the dictionaries
image_urls = []
items = hemi_soup.findAll('div', class_='item')

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys `img_url` and `title`.
#Append the dictionary with the image url string and the hemisphere title to a list. 
# This list will contain one dictionary for each hemisphere.
for item in items:
    image_url_dict = {'title': [], 'img_url': [],}
    title = item.find('h3').text
    #print(img_url_dict)
    
    hemi_img_url = browser.find_link_by_partial_text('Sample')['href']
    img_url_dict = {'title': title, 'img_url': hemi_img_url}
    #print(image_urls)
    
    image_urls.append(img_url_dict)
    
    
image_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/facilities/flynn-creek-crater-sample-collection'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/facilities/flynn-creek-crater-sample-collection'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/facilities/flynn-creek-crater-sample-collection'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/facilities/flynn-creek-crater-sample-collection'}]